# Derek Lamb

# Summary of the SuSiE method

The sum of single effects (SuSiE) method to identify . While prediction-focused methods, such as LASSO, are capable of handling a set highly correlated variables by arbitrarily discarding all but one of the set without impact on prediction error, this approach is not satisfying for answering causal questions. SuSiE is a variable selection approach that is based upon the Iterative Bayesian Stepwise Selection (IBSS) algorithm, and identifies a subset(s) of genetic variables that contain effect variables (causal factors). This algorithm uses single effects regression (SER) for a number of hypothesized effect variables (L) and estimates the posterior mean effect for the set of variables. The L effects are considered together, and this process is repeated until a convergence threshold is met. Two advantages of SuSiE compared to other methods, such as Bayesian variable selection in regression (BVSR) is that L can be specified by the user, and that SuSiE constructs a number of credible sets (at most L) that are likely to contain these effect variables. These sets can be further scrutinized to efficiently identify causal relationships between genetic variables and phenotypes.

# Overview

This fine-mapping tutorial was developed using simulated data shipped with [`susieR`](https://github.com/stephenslab/susieR) package.
The simulated data contains simulated phenotype matrix $Y$ in $N \approx 600$ individuals. 
The goal is to identify from the genotype matrix $X_{N\times P}$ ($P=1000$) the genetic variables that directly cause changes in the phenotypic values.

**There are several questions throughout the notebook, and please answer each of them by adding cells (in Markdown or R) under the question and keep the code you use there as well.**

The simulated data-set is [available
here](https://github.com/stephenslab/susieR/blob/master/data/N3finemapping.RData),
as part of the `susieR` package. In this tutorial we will first explore the data with conventional univariate association analysis,
then perform preliminary fine-mapping analysis under simplified 
assumptions to illustrate the idea of statistical fine-mapping, and finally use the R package `susieR` to perform formal Bayesian fine-mapping analysis.
We will also discuss fine-mapping using association summary statistics.

# Explore the association data-set

First let's install the package `susieR` according to [its Github repo](https://github.com/stephenslab/susieR) and `corrplot` to draw some plots later. This may take a while.

In [1]:
install.packages("susieR")
install.packages("corrplot")

also installing the dependency ‘RcppArmadillo’


Warning message in install.packages("mixsqp"):
“installation of package ‘RcppArmadillo’ had non-zero exit status”
Warning message in install.packages("mixsqp"):
“installation of package ‘mixsqp’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘RcppArmadillo’, ‘mixsqp’


Warning message in install.packages("susieR"):
“installation of package ‘RcppArmadillo’ had non-zero exit status”
Warning message in install.packages("susieR"):
“installation of package ‘mixsqp’ had non-zero exit status”
Warning message in install.packages("susieR"):
“installation of package ‘susieR’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



Please execute the following code cell to load the simulated data.

In [3]:
rm(list=ls())
library(susieR)
library(corrplot)
data(N3finemapping)
attach(N3finemapping)

ERROR: Error in library(susieR): there is no package called ‘susieR’


In [ ]:
ls()

The loaded dataset, `N3finemapping`, contains regression data $X$ and $Y$, 
along with some other relevant attributes in the context of genetic studies. It also
contains the "true" regression coefficients the data is simulated from.

In [ ]:
names(N3finemapping)

In [ ]:
dim(X)
head(X)

-----------------------------------------

**Example question: in the genotype matrix `X`, how many individuals and the genetic variants are included? What is the genotype value for the 100th variant of the 21st individual?**

**Example answer:** The genotype matrix has 574 individuals and 1001 genetic variants. The genotype value for the 100th variant of the 21st individual is 0.4408.

In [6]:
dim(X)

[1]  574 1001

In [7]:
X[21, 100]

[1] 0.4407666

-----------------------------------------

The phenotype matrix contains values for the 574 individuals for 2 traits,

In [ ]:
dim(Y)

We focus on the first trait, let

In [ ]:
y = Y[,1]
b = true_coef[,1]

There are 3 "causal" variants for this trait,

In [ ]:
which(b != 0)

Note: in this tutorial we will avoid the phrase "causal", and use "effect variable" to refer to the variants having true non-zero effects on the phenotype.

-----------------------------------------

**Question 1: Please use R function `plot` to visualize the true effects vector. What are the effect sizes for these 3 "causal" variants?**

**Answer:**

In [ ]:
plot(b)

Two of the causal variants, 403 and 773, have positive effects between 1.0 and 1.5. The variant 653 has a negative effect, around -0.9

-----------------------------------------

# Association testing via simple linear regression

`univariate_regression` function in `susieR` can be used to perform association testing 
by fitting univariate simple regression for each variable separately.
The results are $\hat{\beta}$ for effect size estimates, and their standard errors $SE(\hat{\beta})$.

In [ ]:
sumstats <- univariate_regression(X, y)

-----------------------------------------

**Question 2: Please use R function plot to compare the true effects variable `b` with the estimated effect sizes `sumstats$betahat`. Are they consistent, and why?**

**Answer:**

In [ ]:
plot(sumstats$betahat, b)

The values for the three true effects are consistent; however, the null effects estimated by univariate regression are highly variable, and rarely 0.

-----------------------------------------

z-scores can be derived from these quantities; $-log_{10}$ p-values can then becomputed from z-scores, 

In [ ]:
z_scores <- sumstats$betahat / sumstats$sebetahat
log10p <- -log10(pchisq(z_scores^2,1,lower.tail=F))

-----------------------------------------

**Question 3: How many variables have p-values smaller than $5 \times 10^{-8}$? How would you interpret those variants based on the univariate association analysis?**


**Answer:**

In [ ]:
plot(log10p)
abline(h = -log(5e-8, 10), col=2)

There are 8 variables with p-values smaller than 5e-8. Based upon univariate regression alone, I would say that these loci are highly correlated with the phenotype.

**Question 4: What's the correlation between these variables (with p-values smaller than $5 \times 10^{-8}$) in the original genotype data `X`? Draw a correlogram and print out the pair with highest positive correlation. What does it mean for those two variants?**

**Answer:**

In [ ]:
loci_sig = which(log10p >7.3)
X[,loci_sig] |> 
  cor() |> 
  corrplot(method = "number")

Genetic variables 4 and 6 are the most strongly correlated ($\rho = 0.98$). This means that knowing information about just one of these variables is highly informative in understanding the other.

-----------------------------------------

`susieR` package provides a plot function to generate Manhattan plot for the association test results. Notice that the true effect variables are labelled in red (true effects provided to `susie_plot` via setting `b=` parameter),

In [ ]:
susie_plot(z_scores,y="z",b=b)

As shown on this plot, the "lead" SNP (the one with smallest p-value) is indeed one of the true effect variables. However, the 2nd most significant SNP is not one of the other two effect variables, and there are many other SNPs having p-values smaller than $5 \times 10^{-8}$, a widely accepted significant level threshold for genetic association studies.

-----------------------------------------

**Question 5: What index is the second most significant SNP? Is it one of the other two true effect variables? What would you say about the relationship between the top and 2nd most significant SNPs?**


**Answer:**

In [ ]:
which(log10p == sort(log10p)[1000])

Index 777 is the second most significant SNP. This is not one of the effect variables, in fact it is the 6th variable in the correlogram above. It is highly correlated with index 773, the 4th variable in the correlogram and one of the true effect variables.

-----------------------------------------

# "One causal variant" assumption for fine-mapping

Under the assumption that there is only one true effect in the region of interest, fine-mapping does not have to use linkage disequilibrium (LD) information between variables. It can be performed by simply calculating the likelihood of the association model for each variable separately, then computing the probability for a variable being the true effect as the ratio of its likelihood over the sum of likelihoods over all variables. We use linear regression `lm` in R to compute the log-likelihood,

In [ ]:
lliks = sapply(1:ncol(X), function(i) logLik(lm(y~X[,i])))

Then compute the probability for each variable being the true effect variable: 

In [ ]:
lliks = lliks - max(lliks) # To avoid taking exp on large numbers in the next line, thus improving numerical stability 
probs = exp(lliks)/sum(exp(lliks))

We use `susie_plot` function to plot these probabilities computed, and color label the true effect variables:

In [ ]:
susie_plot(probs, y="PIP", b=b, ylab = "Prob. true effect (assuming one effect variable)")

As expected, under the "one causal variant" assumption, only the lead SNP is identified with over 80% probability of having a non-zero effect. The other two true effect variables in red are not identified.

## 95% credible set for single effect

95% credible set (CS) is defined as a set of variables with 95% probability of containing a true effect. By visually examine the plot above, we can see that the lead SNP has over 80% probability being the true effect, and another SNP near it has almost 20% probability of being the true effect. The 95% credible set should thus contain both of them in order for the coverage to exceed 95%. Indeed, we can formally verify this using an internal function `in_CS` to assess if a variable is in a single effect CS. We report the indices of these variables,

In [2]:
which(susieR:::in_CS(t(probs), coverage = 0.95) != 0)

ERROR: Error in loadNamespace(x): there is no package called ‘susieR’


These variables indeed correspond to the two non-zero probability variables on the plot above.

# Fine-mapping with individual level data using Sum of Single Effects model

When multiple true effects are expected, formal statistical fine-mapping via Bayesian Variable Selection
should be performed. We fit a Sum of Single Effects (SuSiE) model implemented in `susieR` package, 
assuming at most 10 causal variables (although SuSiE is robust to the choice of `L`).

In [ ]:
fitted <- susie(X, y, L = 10)

## 95% credible sets for each single effect

By default, `susie` function computes 95% CS each containing one effect variable,

In [ ]:
print(fitted$sets)

The 3 true signals have been captured by the 3 CS reported here. The
3rd CS contains many variables, including the true causal variable
`403`. The minimum absolute correlation is 0.86.

-----------------------------------

**Question 6: Print the credible sets if we use 99% as the coverage. Hint: you may want to check the source code for `susie` function from [here](https://github.com/stephenslab/susieR)**.

**Answer:**

In [ ]:
fit99 <- susie(X, y, L = 10, coverage = 0.99)

print(fit99$sets)

**Question 7: does higher coverage mean you have larger CS, or smaller CS? Why is that? Please look at the 3rd CS as an example in the two fitted results, and explain why the minimum absolute correlation is lower when the coverage is higher?**

**Answer:**

Higher coverage means that you may have a larger CS. Taking the 3rd CS as an example, it required 27 loci to reach a 95% probability of containing a true effect. When I increased the coverage to 99%, these 27 variables were insufficient, and so the set needed to increase to 35 variables to ensure that there was a 99% chance that a true effect was within the set. 

The minimum absolute correlation decreases, because the 8 variables thater were added to increase the coverage have smaller absolute correlations than the initial variables.

-----------------------------------

## Posterior inclusion probabilities

Here we summarize the fine-mapping results by posterior inclusion
probability (PIP):

In [ ]:
susie_plot(fitted, y="PIP", b=b, add_legend=T)

The true effect variables are colored red, and the three 95% CS identified are
circled in different colors. 

-------------------------------------------

**Question 8: Recall the true effect variables we know before running any susie, and for the variable 403, what is the PIP? What's the average PIP of the 3rd CS? Is the PIP for variable 403 higher or lower than the average, and why is that?**

Hint: You can use the code below to compare $-log_{10}$ p-value and PIP for the CS containing variable 403:

```R
    i  <- fitted$sets$cs$L3
    z3 <- cbind(i,log10p[i],fitted$pip[i])
    colnames(z3) <- c('position', '-log10 pvalue', 'PIP')
    z3[order(z3[,2], decreasing = TRUE),]
```

**Answer:**

In [ ]:
# pip for 403
fitted$pip[403]

# mean pip for 3rd CS
mean(fitted$pip[fitted$sets$cs$L3])

The PIP for 403 is lower than the average (0.032 vs 0.035). The SNPs with higher PIPs tend to have higher p-values, but this isn't strictly true. While these variables are correlated with one another, some of the other variables, due to random chance, may have had stronger associations with the phenotype in this sample.

***

**Question 9: Please rerun the `susie` analysis setting `L=1` to detect a maximum of only one effect across the region. How does the result compare to the previous section of "one causal effect" fine-mapping using likelihoods from univariate analysis?**

**Answer:**

In [ ]:
l1_fit <- susie(X, y, L = 1)

print(l1_fit$sets)

susie_plot(l1_fit, y="PIP", b=b)

The result setting L=1 is identical to the prior results from "one causal effect" fine mapping. 

-------------------------------------------

## Choice of prior effect size

In SuSiE, the prior effect size is parameterized by the percentage of variance explained by a variable, $h_j^2$. 
By default, SuSiE estimates $h_j^2$ and uses it as the prior (an empirical Bayes approach). 
Here we demonstrate fixing the prior effect size to 0.2, i.e., we believe a non-zero effect variable can explain 20% variation in quantitative trait values:

In [ ]:
fitted2 = susie(X, y, L = 10, estimate_prior_variance = FALSE, scaled_prior_variance = 0.2)
susie_plot(fitted2, y='PIP', b=b, add_legend=T)

The result is very similar to when prior effects are estimated. 

-------------------------------------------

**Question 10: Please change `scaled_prior_variance` to different numbers between 0 and 1, and examine the results. What happens if you set it to 0.001, and why?**

**Answer:**

In [ ]:
fitted3 = susie(X, y, L = 10, estimate_prior_variance = FALSE, scaled_prior_variance = 0.8)
susie_plot(fitted3, y='PIP', b=b, add_legend=T)

fitted4 = susie(X, y, L = 10, estimate_prior_variance = FALSE, scaled_prior_variance = 0.5)
susie_plot(fitted4, y='PIP', b=b, add_legend=T)

fitted5 = susie(X, y, L = 10, estimate_prior_variance = FALSE, scaled_prior_variance = 0.01)
susie_plot(fitted5, y='PIP', b=b, add_legend=T)

fitted6 = susie(X, y, L = 10, estimate_prior_variance = FALSE, scaled_prior_variance = 0.001)
susie_plot(fitted6, y='PIP', b=b, add_legend=T)

The choice of prior here reflects the maximum strength of association we're initially predicting in the model. A value of 0.2 already captures the true effect well enough that increasing `scaled_prior_variance` to 0.5 or 0.8 has a negligable effect on the model. Decreasing it, even to 0.01, also has little effect, but when it is decreased below 0.003, the model is forced to decrease the posterior probility, and thus shift that probability onto other loci. So the PIP for some variables gets reduced, and that probability is transfered to other correlated variables.

-------------------------------------------

# Fine-mapping with summary statistics via `susie_rss`

Summary statistics of genetic association studies typically contain effect size ($\hat{\beta}$ coefficient from regression), p-value and minor allele frequencies. 
These information can be used to perform fine-mapping with given an additional input of correlation matrix between variables.
The correlation matrix in genetics is typically referred to as LD matrix (LD for linkage disequilibrium). 
One may use external reference panels to estimate it when this matrix cannot be obtained from samples directly. There is a potential problem using LD from reference panel, and we demonstrated it in the next section.
Caution that LD matrix here has to be correlation matrix $r$, not $r^2$ or $|r|$.

We have previously computed z-scores for this data-set. The correlation matrix can be computed directly from $X$ matrix, 

In [ ]:
R <- cor(X)

SuSiE regression with summary statistics is implemented as `susie_rss` function, 

In [ ]:
fitted_rss <- susie_rss(z_scores, R, L = 10)

We compare the PIP obtained with previous analysis using individual level data,

In [ ]:
plot(fitted$pip, fitted_rss$pip, ylim=c(0,1))

The results very much agree. This is expected when the summary statistics are indeed derived from the individual level data.

## Use LD from external reference data

When original genotype information is not available, 
one may use reference panel to estimate LD matrix. 
False discoveries may occur if the LD pattern from the reference panel does not agree well with the original sample. 

To illustrate, we randomly generated 500 samples from $N(0,R)$ and treated them as reference panel genotype matrix `X_ref`.

In [ ]:
set.seed(1234)
tmp = matrix(rnorm(500*1001), 500, 1001)
eigenR = eigen(R)
eigenR$values[eigenR$values < 1e-10] = 0
X_ref = tmp %*% (eigenR$values * t(eigenR$vectors))
R_ref = cor(X_ref)

Using `susie_rss` function with LD from reference panel,

In [ ]:
fitted_rss_ref <- susie_rss(z_scores, R_ref, L = 10)
susie_plot(fitted_rss_ref, y="PIP", b=b)

-------------------------------------------

**Question 11: Please compare and comment on the CS reported using reference LD panel versus using the correct LD computed from genotype data. Are there any false discoveries?**

**Answer:**

There are several false discoveries and indeed no true discoveries. Rather than the three correct CS generated before, now there are 10. There appears to be inflation of some of the PIPs, several are very close to/exactly 1.

-------------------------------------------

We recommend the following modification to the LD matrix when reference panel ($R_{ref}$) are used,
$$
\tilde{R} = \text{cov2cor}((1-w)R_{ref}+ w zz^\intercal)),
$$
where z is the original summary statistics. We suggest using $w = 1/n_{ref}$, $n_{ref}$ is the sample size from the reference panel. The modified LD matrix becomes
$$
\tilde{R} = \text{cov2cor}(\frac{n_{ref}-1}{n_{ref}}R_{ref}+ \frac{1}{n_{ref}} zz^\intercal)) = \text{cov2cor}(\frac{1}{n_{ref}}(X_{ref}^\intercal X_{ref}+ zz^\intercal)),
$$
where `X_ref` is column-standardized. This modification is implemented in `susie_rss` with parameter `z_ld_weight`:

In [ ]:
n_ref = 500
fitted_rss_ref_corrected <- susie_rss(z_scores, R_ref, z_ld_weight = 1/n_ref, L = 10)
susie_plot(fitted_rss_ref_corrected, y="PIP", b=b)

***
**Question 12: Please compare and comment on the results using adjusted reference LD panel. Is the result improved?**

**Answer:**

The result is improved from the prior one, but not back to the original susie result. There are now only two CS, but both of them contain the effect variables. SNP 653 is lost, and there are a few loci that have high PIPs but are not included in any CS.

***

# Exercise: fine-mapping analysis on a different data-set

Please load and analyze another data-set provided in `susieR` package:

In [ ]:
rm(list=ls())
library(susieR)
data(N2finemapping)

Please use the same R code for `N3finemapping` data to analyze this new data-set, similarly focusing on the first column of the phenotype matrix as the phenotype of interest. 
This data-set contains two true effect variables. However, as you are soon to find out in univariate linear regression analysis, **the lead SNP via univariate analysis is neither of the two effect variables**.

-------------------------------------------

**Question 13: please run `susie_rss` on the N2finemapping dataset and report the results below. Does it capture any true effect variant?**

**Answer:**

In [ ]:
# define variables
X = N2finemapping$X
y = N2finemapping$Y[,1]
b = N2finemapping$true_coef[,1]

# identify true effects
which(b != 0)

# fit univariate
uni_N2 <- univariate_regression(X, y)

# plot univariate results
plot(uni_N2$betahat, b)

z_scores <- uni_N2$betahat / uni_N2$sebetahat
log10p <- -log10(pchisq(z_scores^2,1,lower.tail=F))

plot(log10p)

The univariate analysis on the `N2finemapping` data set is less productive than the examples above. While several p-values are in the 2-3 range, this is unremarkable given the number of tests being done. No p-values stand out below 10e-7 as before.

In [ ]:
n2_susie <- susie_rss(z_scores, cor(X), n = 600, L = 10)

plot(n2_susie$pip, ylim=c(0,.1))

The true effects here are at indexes 337 and 999. Neither are captured in this analysis. The `susie` function also failed to generate credible sets containing these loci. This may be due to a lower strength of association in this data set.

# Notes
SuSiE is robust to overstating L. When I specified L = 10, when there were only 3 true effects, it successfully identified only 3 sets.

-------------------------------------------

In [ ]:
sessionInfo()